In [ ]:
from google.colab import files
import zipfile
import os
import re
import pandas as pd
from tqdm.notebook import tqdm
import concurrent.futures

def extraire_infos_medline(fichier_path, nom_fichier_sans_ext):
    with open(fichier_path, 'r', encoding='utf-8') as f:
        contenu = f.read()

    articles = re.split(r'\n(?=PMID-)', contenu)
    infos = []

    for article in articles:
        pmid = re.search(r'PMID-\s*(\d+)', article)
        ti = re.search(r'TI\s+-\s+(.*?)(?=\n[A-Z]{2,4} -|\Z)', article, re.DOTALL)
        ab = re.search(r'AB\s+-\s+(.*?)(?=\n[A-Z]{2,4} -|\Z)', article, re.DOTALL)
        ad = re.findall(r'AD\s+-\s+(.*?)(?=\s*\n(?:FAU\s*-\s*|\w{2,4}\s*-\s*)|\Z)', article, re.DOTALL)
        edat = re.search(r'EDAT-\s*(\d{4}/\d{2}/\d{2})', article)

        infos.append({
            'Nom du fichier': nom_fichier_sans_ext,
            'PMID': pmid.group(1) if pmid else None,
            'Titre (TI)': ti.group(1).replace('\n', ' ').strip() if ti else None,
            'Résumé (AB)': ab.group(1).replace('\n', ' ').strip() if ab else None,
            'Adresse (AD)': ' | '.join([a.replace('\n', ' ').strip() for a in ad]) if ad else None,
            'Date EDAT': edat.group(1) if edat else None
        })

    return infos

def process_file(chemin_fichier):
    nom_fichier_sans_ext = os.path.splitext(os.path.basename(chemin_fichier))[0]
    infos = extraire_infos_medline(chemin_fichier, nom_fichier_sans_ext)
    if infos:
        df = pd.DataFrame(infos)
        excel_path = os.path.join(excel_dir, nom_fichier_sans_ext + ".xlsx")
        df.to_excel(excel_path, index=False)
        return excel_path
    return None

try:
    # Étape 1 : Téléversement du ZIP
    uploaded = files.upload()
    zip_filename = list(uploaded.keys())[0]

    # Étape 2 : Extraction dans un dossier temporaire
    extract_dir = "temp_extraction"
    os.makedirs(extract_dir, exist_ok=True)

    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

    # Étape 3 : Recherche des fichiers .txt
    fichiers_txt = [
        os.path.join(root, file)
        for root, _, files_in_dir in os.walk(extract_dir)
        for file in files_in_dir
        if file.endswith(".txt")
    ]

    if not fichiers_txt:
        print("❌ Aucun fichier .txt trouvé dans le ZIP.")
    else:
        print(f"✅ {len(fichiers_txt)} fichier(s) .txt trouvé(s). Traitement en cours...")

        excel_dir = "resultats_excel"
        os.makedirs(excel_dir, exist_ok=True)

        # Utilisation de ThreadPoolExecutor pour le traitement parallèle
        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = [executor.submit(process_file, fichier) for fichier in fichiers_txt]
            excel_files_paths = [future.result() for future in tqdm(concurrent.futures.as_completed(futures), total=len(fichiers_txt), desc="Traitement des fichiers") if future.result() is not None]

        # Étape 4 : Création du ZIP des fichiers Excel
        zip_result_name = os.path.splitext(zip_filename)[0] + "_excels.zip"
        with zipfile.ZipFile(zip_result_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for filepath in excel_files_paths:
                if filepath:
                    zipf.write(filepath, arcname=os.path.basename(filepath))

        # Étape 5 : Téléchargement
        print(f"✅ Fichiers Excel générés : {len(excel_files_paths)}")
        files.download(zip_result_name)

except Exception as e:
    print(f"❌ Une erreur est survenue : {e}")


Saving pubmed_fichier.zip to pubmed_fichier.zip
✅ 54 fichier(s) .txt trouvé(s). Traitement en cours...


Traitement des fichiers:   0%|          | 0/54 [00:00<?, ?it/s]

✅ Fichiers Excel générés : 54


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>